In [6]:
# Importing essential libraries
import gc
import os
import random
import warnings
import numpy as np
import pandas as pd
from IPython.display import display

# PyTorch for deep learning
import timm
import torch
import torch.nn as nn  
import torch.optim as optim
import torch.nn.functional as F

# torchvision for image processing and augmentation
import torchvision.transforms as transforms

# Suppressing minor warnings to keep the output clean
warnings.filterwarnings('ignore', category=Warning)

# Reclaim memory no longer in use.
gc.collect()

808

In [7]:
class Config:
    seed=42
    image_transform=transforms.Resize((512, 512))
    num_folds=4

In [11]:
models = []
# Load EfficientNetB0
for j in range(Config.num_folds+1):
    # Create the same model architecture as during training
    model_effnet_b0 = timm.create_model('efficientnet_b0', pretrained=False, num_classes=6, in_chans=1)
    # Load the trained weights from the corresponding file
    model_effnet_b0.load_state_dict(torch.load(f'/kaggle/input/raww/pytorch/raww/1/efficientnet_b0_fold{j}.pth', map_location=torch.device('cpu')))
    # Append the loaded model to the models list
    models.append(model_effnet_b0)
gc.collect()


13813

In [ ]:
# Load test data and sample submission dataframe
test_df = pd.read_csv("/kaggle/input/hms-harmful-brain-activity-classification/test.csv")
submission = pd.read_csv("/kaggle/input/hms-harmful-brain-activity-classification/sample_submission.csv")

# Merge the submission dataframe with the test data on EEG IDs
submission = submission.merge(test_df, on='eeg_id', how='left')

# Generate file paths for each spectrogram based on the EEG data in the submission dataframe
submission['path'] = submission['spectrogram_id'].apply(lambda x: f"/kaggle/input/hms-harmful-brain-activity-classification/test_spectrograms/{x}.parquet")

# Display the first few rows of the submission dataframe
display(submission.head())

# Reclaim memory no longer in use
gc.collect()

In [ ]:
# Get file paths for test spectrograms
paths = submission['path'].values
test_preds = []

# Generate predictions for each spectrogram using all models
for path in paths:
    eps = 1e-6
    # Read and preprocess spectrogram data
    data = pd.read_parquet(path)
    data = data.fillna(-1).values[:, 1:].T
    data = np.clip(data, np.exp(-6), np.exp(10))
    data = np.log(data)
    
    # Normalize the data
    data_mean = data.mean(axis=(0, 1))
    data_std = data.std(axis=(0, 1))
    data = (data - data_mean) / (data_std + eps)
    data_tensor = torch.unsqueeze(torch.Tensor(data), dim=0)
    data = Config.image_transform(data_tensor)

    test_pred = []
    # Generate predictions using all models
    for model in models:
        model.eval()
        with torch.no_grad():
            pred = F.softmax(model(data.unsqueeze(0)))[0]
            pred = pred.detach().cpu().numpy()
        test_pred.append(pred)
        
    test_preds_mean = np.mean(test_pred, axis=0)    
    test_preds.append(test_preds_mean)

# Convert the list of predictions to a NumPy array for further processing
test_preds = np.array(test_preds)

# Reclaim memory no longer in use
gc.collect()

In [ ]:
# Load the sample submission file and update it with model predictions for each label
submission = pd.read_csv("/kaggle/input/hms-harmful-brain-activity-classification/sample_submission.csv")
labels = ['seizure', 'lpd', 'gpd', 'lrda', 'grda', 'other']

# Assign model predictions to respective columns in the submission DataFrame
for i in range(len(labels)):
    submission[f'{labels[i]}_vote'] = test_preds[:, i]

# Save the updated DataFrame as the final submission file
submission.to_csv("submission.csv", index=None)

# Display the first few rows of the submission file
display(submission.head())

# Reclaim memory no longer in use.
gc.collect()